#**ensembleLearning2**


Ensemble Learning
Ensemble Learning adalah proses di mana beberapa model, seperti pengklasifikasi atau ahli, secara strategis dihasilkan dan digabungkan untuk memecahkan kecerdasan komputasi tertentu. Ensemble Learning utamanya digunakan untuk meningkatkan (klasifikasi, prediksi, perkiraan fungsi, dll.) kinerja model, atau mengurangi kemungkinan pemilihan model yang buruk.

**pertama Install library yang di butuhkan**

In [50]:
%%capture
!pip install snscrape
!pip install pandas
!pip install Sastrawi
!pip install scikit-learn

## **Menulis Script Konfigurasi Snscrape**

In [51]:

import snscrape.modules.twitter as sntwitter
import pandas as pd
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

search_query = "dedi mulyadi"
jumlah_tweets = 100
tweets = []


for tweet in sntwitter.TwitterSearchScraper(search_query).get_items():
    
    if len(tweets) == jumlah_tweets:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, 'None'])
        
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'Label'])
print(df)


<ipython-input-51-d4f7d2c62137>:16: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, 'None'])


                        Date             User  \
0  2022-12-13 16:53:15+00:00       aprella234   
1  2022-12-13 12:44:21+00:00     info_daratan   
2  2022-12-13 11:19:09+00:00      abcdefuckyu   
3  2022-12-13 09:30:47+00:00        kompascom   
4  2022-12-13 08:52:40+00:00      suaradotcom   
..                       ...              ...   
95 2022-12-09 05:14:59+00:00    newmindarmy45   
96 2022-12-09 04:36:19+00:00  HumasPolresMaj1   
97 2022-12-09 03:53:27+00:00    OposisiCerdas   
98 2022-12-09 03:32:30+00:00         geloraco   
99 2022-12-09 03:32:06+00:00         geloraco   

                                                Tweet Label  
0   berasa ditampar sama ucapan kang Dedi Mulyadi:...  None  
1   Dedi Mulyadi tiba-tiba berbelok arah menuju Ru...  None  
2   Jika benar nanti RK masuk golkar untuk di capr...  None  
3   Perseteruan sejoli yang batal menikah karena p...  None  
4   Dijodohkan Netizen dengan Dedi Mulyadi, Bupati...  None  
..                                     

### Melakukan Pengecekan Berkas Hasil Scrape Lama

Karena setelah melakukan scraping data tweet kita perlu memberikan label secara manual, maka untuk menghindari berkas lama tertimpa oleh berkas baru, disini kita akan melakukan pengecekan berkas hasil scrape lama, apakah ada pada direktori atau tidak. Jika berkas hasil scrape lama tidak ditemukan pada direktori, maka hasil scrape sebelumnya (yang ada pada Pandas Dataframe) akan diespor menjadi berkas csv.


In [52]:
%%capture
import os



output_stream = os.popen('tweets_labeled.csv')
res = output_stream.read()
if res == '':
  df.to_csv('tweets_labeled.csv')

## **Import Hasil**

In [53]:
import pandas as pd


# pd.options.mode.chained_assignment = None
# pd.options.display.max_colwidth = None
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None
df = pd.read_csv('tweets_labeled.csv', usecols=['User', 'Tweet', 'Label'])
df

,User,Tweet,Label
0,aprella234,berasa ditampar sama ucapan kang Dedi Mulyadi:...,None
1,info_daratan,Dedi Mulyadi tiba-tiba berbelok arah menuju Ru...,None
2,abcdefuckyu,Jika benar nanti RK masuk golkar untuk di capr...,None
3,kompascom,Perseteruan sejoli yang batal menikah karena p...,None
4,suaradotcom,"Dijodohkan Netizen dengan Dedi Mulyadi, Bupati...",None
...,...,...,...
95,newmindarmy45,Kang Dedi Mulyadi Gelisah Kelakuan Ambu Anne K...,None
96,HumasPolresMaj1,Dalam upaya menangkal masuknya paham radikalis...,None
97,OposisiCerdas,Kang Dedi Mulyadi Gelisah Kelakuan Ambu Anne K...,None
98,geloraco,Nama Bupati Purwakarta dan Dedi Mulyadi Kian P...,None


## **Preprocessing Data Tweets**

In [54]:
import string

#Mengapus link dan mention
df['Tweet'] = df['Tweet'].replace(r'\s+',' ', regex=True)
indx = 0
for i in df['Tweet']:
  temp = df['Tweet'][indx].split()
  for j in temp:
    if 'http' in j:
      df['Tweet'] = df['Tweet'].replace(r'%s'%j," ", regex=True)
    if '@' in j:
      df['Tweet'] = df['Tweet'].replace(r'%s'%j," ", regex=True)
  indx+=1



#mengubah menjadi huruf kecil
df['Tweet'] = df['Tweet'].str.lower()

#menghapus tanda baca
for char in string.punctuation:
    df['Tweet'] = df['Tweet'].replace(r'[\%s]'%char," ", regex=True)

#menghapus angka
df['Tweet'] = df['Tweet'].replace(r'\d+',' ', regex=True)

#menghapus karakter kosong
df['Tweet'] = df['Tweet'].replace(r'\s+',' ', regex=True)
df['Tweet'][2]



'jika benar nanti rk masuk golkar untuk di capres cawapreskan oleh kib tapi menurutku bukan itu tujuan utama ridwan kamil tetapi bagaimana suara golkar di jabar aman karena saingan terberat rk di pilgub nanti dedi mulyadi nanti rk akan berpasangan dengan bima arya opinipolitik'

### Stopwords Removal

In [55]:
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/smilesense/stopwords-id/master/stopwords-id.json") as list_stopwords:
    data_stopword = json.load(list_stopwords)

for i in data_stopword:
    df['Tweet'] = df['Tweet'].replace(r'\b%s\b'%i, '', regex=True)
df['Tweet'] = df['Tweet'].replace(r'\s+',' ', regex=True)

df['Tweet'][2]

' rk masuk golkar capres cawapreskan kib menurutku tujuan utama ridwan kamil suara golkar jabar aman saingan terberat rk pilgub dedi mulyadi rk berpasangan bima arya opinipolitik'

### Stemming

In [56]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# stemming process

try:
  ind = 0
  for sentence in df['Tweet']:
      df['Tweet'][ind] = stemmer.stem(str(sentence))
      ind+=1
  raise KeyboardInterrupt
except KeyboardInterrupt:
  print(df['Tweet'][2])
else :
  print(df['Tweet'][2])


rk masuk golkar capres cawapreskan kib turut tuju utama ridwan kamil suara golkar jabar aman saing berat rk pilgub dedi mulyadi rk pasang bima arya opinipolitik


### Tokenisasi

In [57]:
indeks = 0
df2 = df.copy()
for tweet in df2['Tweet']:
    df2['Tweet'][indeks] = str(tweet).split()
    indeks+=1

df2['Tweet'][2]


['rk',
 'masuk',
 'golkar',
 'capres',
 'cawapreskan',
 'kib',
 'turut',
 'tuju',
 'utama',
 'ridwan',
 'kamil',
 'suara',
 'golkar',
 'jabar',
 'aman',
 'saing',
 'berat',
 'rk',
 'pilgub',
 'dedi',
 'mulyadi',
 'rk',
 'pasang',
 'bima',
 'arya',
 'opinipolitik']

### Hasil Preprocessing


In [58]:
df2

,User,Tweet,Label
0,aprella234,"[asa, tampar, ucap, kang, dedi, mulyadi, orang...",None
1,info_daratan,"[dedi, mulyadi, belok, arah, rumah, sakit, has...",None
2,abcdefuckyu,"[rk, masuk, golkar, capres, cawapreskan, kib, ...",None
3,kompascom,"[seteru, joli, batal, meni, minta, mas, kawin,...",None
4,suaradotcom,"[jodoh, netizen, dedi, mulyadi, bupati, karawa...",None
...,...,...,...
95,newmindarmy45,"[kang, dedi, mulyadi, gelisah, laku, ambu, ann...",None
96,HumasPolresMaj1,"[upaya, tangkal, masuk, paham, radikalisme, te...",None
97,OposisiCerdas,"[kang, dedi, mulyadi, gelisah, laku, ambu, ann...",None
98,geloraco,"[nama, bupati, purwakarta, dedi, mulyadi, kian...",None


## **Term Frequency (TF)**

### Term Frequency Keseluruhan

In [59]:
listkata = []
for tweet in df['Tweet']:
    listkata = listkata + str(tweet).split()

listkata_nodup = list(dict.fromkeys(listkata))
hasil_hitung = {}
for test1 in listkata_nodup:
    jumlah = 0
    for test2 in range(len(listkata)):
        if test1 == listkata[test2]:
            jumlah+=1
    hasil_hitung.update({'%s'%test1 : jumlah})
    
hasil_hitung = dict(sorted(hasil_hitung.items(), key=lambda item: item[1], reverse=True))
print(hasil_hitung)

{'dedi': 94, 'mulyadi': 93, 'anne': 35, 'bupati': 21, 'rumah': 19, 'bandung': 17, 'ambu': 17, 'purwakarta': 17, 'ratna': 15, 'kang': 14, 'hasan': 14, 'mustika': 14, 'belok': 13, 'arah': 13, 'sakit': 13, 'sadikin': 13, 'golkarsolid': 13, 'jodoh': 10, 'nikah': 10, 'karawang': 9, 'ryan': 9, 'cerai': 9, 'sertifikat': 8, 'yessy': 8, 'dono': 8, 'aja': 7, 'mahar': 7, 'biaya': 7, 'cellica': 6, 'gagal': 6, 'nafkah': 6, 'tuduh': 6, 'laku': 6, 'orang': 5, 'jabar': 5, 'buka': 5, 'hati': 5, 'sosok': 5, 'populer': 5, 'longsor': 5, 'relokasi': 5, 'keluarga': 5, 'korban': 5, 'nama': 5, 'gelisah': 5, 'dukun': 5, 'tradisi': 5, 'kalo': 4, 'rk': 4, 'golkar': 4, 'batal': 4, 'damai': 4, 'pintu': 4, 'anak': 4, 'nya': 4, 'gugat': 4, 'istri': 4, 'kian': 4, 'jelang': 4, 'milu': 4, 'ngajeng': 4, 'pesan': 4, 'pasang': 3, 'bima': 3, 'arya': 3, 'anggota': 3, 'dpr': 3, 'ri': 3, 'mudah': 3, 'bikin': 3, 'bareng': 3, 'rupiah': 3, 'dedimulyadi': 3, 'ya': 3, 'nurachdiana': 3, 'wanita': 3, 'ga': 3, 'maju': 3, 'tohok': 3, 

### Term Frequency Tiap Tweet

In [60]:
def terms(dataframe):
  # pd.options.mode.chained_assignment = None
  # pd.options.display.max_colwidth = None
  # pd.options.display.max_columns = None
  # pd.options.display.max_rows = None

  df3 = dataframe.copy()
  a = 1
  for inter in range(len(df3['Tweet'])):
      for fitur in hasil_hitung:
          df3['%s'%(fitur)] = 0
          a+=1

  for inter in range(len(df3['Tweet'])):
    for fitur in hasil_hitung:
        cek = df3['Tweet'][inter]
        jumlah = 0
        for iter2 in range(len(cek)):
          if fitur == cek[iter2]:
            jumlah+=1
        df3['%s'%fitur][inter] = jumlah
        a+=1
  df3.to_csv(r'my_data.csv', index=False)
  return df3
terms(df2)



<ipython-input-60-317fe5facf89>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df3['%s'%(fitur)] = 0
<ipython-input-60-317fe5facf89>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['%s'%fitur][inter] = jumlah


,User,Tweet,Label,dedi,mulyadi,anne,bupati,rumah,bandung,ambu,...,aiptu,riyana,laksana,giat,sambang,toga,tokoh,h,wakil,dan
0,aprella234,"[asa, tampar, ucap, kang, dedi, mulyadi, orang...",None,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,info_daratan,"[dedi, mulyadi, belok, arah, rumah, sakit, has...",None,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,abcdefuckyu,"[rk, masuk, golkar, capres, cawapreskan, kib, ...",None,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,kompascom,"[seteru, joli, batal, meni, minta, mas, kawin,...",None,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,suaradotcom,"[jodoh, netizen, dedi, mulyadi, bupati, karawa...",None,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,newmindarmy45,"[kang, dedi, mulyadi, gelisah, laku, ambu, ann...",None,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
96,HumasPolresMaj1,"[upaya, tangkal, masuk, paham, radikalisme, te...",None,1,1,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
97,OposisiCerdas,"[kang, dedi, mulyadi, gelisah, laku, ambu, ann...",None,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
98,geloraco,"[nama, bupati, purwakarta, dedi, mulyadi, kian...",None,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Melakukan Training 

In [61]:
df4 = pd.read_csv('my_data.csv')
df4['Label'].unique()

array(['None'], dtype=object)

In [62]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df4.drop(labels=['Label', 'User', 'Tweet'], axis=1),
    df4['Label'],
    test_size=0.3,
    random_state=0)
y_train

60    None
80    None
90    None
68    None
51    None
      ... 
96    None
67    None
64    None
47    None
44    None
Name: Label, Length: 70, dtype: object

## **Bagging Classifier**

In [63]:
X = df4.drop(labels=['Label', 'User', 'Tweet'], axis=1)
y = df4['Label']

In [64]:
from sklearn import model_selection

from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification

# kfold = model_selection.KFold(n_splits = 3,
#                        random_state = 5, shuffle=True)

X, y = make_classification(n_samples=100, n_features=4,n_informative=2, n_redundant=0,random_state=0, shuffle=True)
clf = BaggingClassifier(base_estimator=SVC(),n_estimators=10, random_state=5).fit(X, y)
clf.predict([[0, 0, 0, 0]])

# results = model_selection.cross_val_score(clf, X, y, cv = kfold)
print("accuracy :")
# print(results.mean())
clf.score(X, y)

accuracy :


0.91

## **Random Forest**

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=4,n_informative=2, n_redundant=0,random_state=0, shuffle=True)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)
# RandomForestClassifier(...)
# print(clf.predict([[0, 0, 0, 0]]))
print("accuracy :")
clf.score(X, y)

accuracy :


0.948

####Naive Bayes

In [66]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
y_pred = clf.fit(X_train, y_train).predict(X_test)
clf.fit(X_train, y_train).score(X_test, y_test)



1.0

## **Stacking Classifier**

In [67]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train).score(X_test, y_test)

1.0

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

from sklearn.tree import DecisionTreeClassifier

estimators = [('rf', RandomForestClassifier(max_depth=2, random_state=0)),('rf1', RandomForestClassifier(max_depth=2, random_state=0)),('rf2', RandomForestClassifier(max_depth=2, random_state=0)),('rf3', RandomForestClassifier(max_depth=2, random_state=0))]
clf = StackingClassifier(estimators=estimators, final_estimator=GaussianNB())
from sklearn.model_selection import train_test_split
print("accuracy :")
clf.fit(X_train, y_train).score(X_test, y_test)